In [1]:
# we are not going to use pickle in the most of case
class GameState(object):
    def __init__(self):
        self.level = 0
        self.lives = 4
        
state = GameState()
state.level += 1
state.lives -= 1

import pickle
state_path = '/tmp/game_state.bin'
with open(state_path, 'wb') as f:
    pickle.dump(state, f)
    
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
print(state_after.__dict__)

{'lives': 3, 'level': 1}


In [2]:
class GameState(object):
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 0
        
state = GameState()
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'points': 0, 'lives': 4, 'level': 0}


In [3]:
# unpickle an old game file, the points attribute is missing!
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
print(state_after.__dict__)

{'lives': 3, 'level': 1}


In [4]:
# the returned object is an instance of the new GameState class 
assert isinstance(state_after, GameState)

In [5]:
# fix the problem with copyreg
import copyreg


class GameState(object):
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points
        
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs,)
        
def unpickle_game_state(kwargs):
    return GameState(**kwargs)
    
copyreg.pickle(GameState, pickle_game_state)

state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'points': 1000, 'lives': 4, 'level': 0}


In [6]:
class GameState(object):
    def __init__(self, level=0, lives=4, points=0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic
    
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'lives': 4, 'level': 0, 'magic': 5, 'points': 1000}


In [7]:
class GameState(object):
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic
        
state_after = pickle.loads(serialized)
print(state_after.__dict__)

TypeError: __init__() got an unexpected keyword argument 'lives'

In [8]:
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1)
    if version == 1:
        kwargs.pop('lives')
        return GameState(**kwargs)
    
copyreg.pickle(GameState, pickle_game_state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'magic': 5, 'points': 1000, 'level': 0}


In [9]:
copyreg.dispatch_table.clear()
state = GameState()
serialized = pickle.dumps(state)
del GameState
class BetterGameState(object):
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

# deserializing an old GameState object will fail
pickle.loads(serialized)

AttributeError: Can't get attribute 'GameState' on <module '__main__'>

In [10]:
print(serialized[:25])

b'\x80\x03c__main__\nGameState\nq\x00)'


In [11]:
# use copyreg for transitioning pickled data to different classes with different names when it's deserialized
copyreg.pickle(BetterGameState, pickle_game_state)
state = BetterGameState()
serialized = pickle.dumps(state)
print(serialized[:35])

b'\x80\x03c__main__\nunpickle_game_state\nq\x00}'
